Set Shots - Expected Miss Model - Model Tuning and Building - GBM

In [1]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/")

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

from expected_score_model.config import set_shots_file_path
# from expected_score_model.domain.preprocessing.data_preprocessor import DataPreprocessor
# from expected_score_model.domain.contracts.mappings import Mappings
from expected_score_model.domain.contracts.modelling_data_contract import ModellingDataContract
from expected_score_model.domain.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from expected_score_model.domain.modelling.supermodel import SuperXGBClassifier
from expected_score_model.domain.modelling.optuna_xgb_param_grid import OptunaXGBParamGrid

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Model Version

In [38]:
model_version = 5
model_name = 'expected_miss_set'
model_file_name = model_name + '_v' + str(model_version)

model_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models"
prediction_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/predictions/"

preprocessor_file_name = 'preprocessor_v' + str(model_version)
preprocessor_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/"

In [39]:
RESPONSE = ModellingDataContract.RESPONSE_MISS

In [40]:
FEATURES = ModellingDataContract.feature_list_set_miss

In [41]:
MONOTONE_CONSTRAINTS = ModellingDataContract.monotone_constraints_set_miss

Load Data

In [42]:
df_modelling = pd.read_csv(set_shots_file_path)
df_modelling.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Behind,Miss,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,GoalTrainingSet,GoalTestSet,GoalValidationSet,BehindTrainingSet,BehindTestSet,BehindValidationSet,MissTrainingSet,MissTestSet,MissValidationSet
11052,208,possGain,behind,1663.0,4,634,683.0,Geelong,Geelong,Tom Hawkins,Tom_Hawkins,Kick,41.0,25.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,0,1,0,1,1,0,105.0,32.0,73.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,41.0,48.0,48.0,26.0,25.0,21.0,21.0,14.0,683.0,644.0,642.0,641.0,49.0,39.0,8.062258,39.0,25.0,46.324939,0.570040,32.660913,0.116867,6.695990,0.364585,20.889203,1521.0,3.663562,True,True,False,False,True,False,False,True,False,False
11053,212,kickIn,goal,1730.0,4,787,885.0,Geelong,Geelong,Patrick Dangerfield,Patrick_Dangerfield,Kick,51.0,43.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,107.0,33.0,74.0,Kick,Contested Mark,Contest Target,Kick Inside 50 Result,51.0,53.0,53.0,53.0,43.0,38.0,38.0,38.0,885.0,852.0,851.0,850.0,98.0,33.0,5.385165,29.0,43.0,51.865210,0.977446,56.003541,0.069260,3.968302,0.214332,12.280314,841.0,3.367296,True,False,False,True,True,False,False,True,False,False
11054,213,centreBounce,goal,1754.0,4,956,979.0,Sydney,Sydney,Paddy McCartin,Paddy_McCartin,Kick,66.0,0.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,0,6,119.0,33.0,86.0,Kick,Free For,Kick Inside 50 Result,Kick Into F50,66.0,74.0,74.0,27.0,0.0,0.0,0.0,17.0,979.0,977.0,968.0,967.0,23.0,2.0,8.000000,14.0,0.0,14.000000,0.000000,0.000000,0.482343,27.636207,2.588598,148.315731,196.0,2.639057,True,True,False,False,True,False,False,False,True,False
11055,219,centreBounce,goal,1846.0,4,1252,1266.0,Sydney,Sydney,Chad Warner,Chad_Warner,Kick,26.0,3.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,0,6,125.0,39.0,86.0,Kick,Uncontested Mark,Ground Kick,Handball,26.0,18.0,-5.0,-2.0,3.0,2.0,-4.0,3.0,1266.0,1265.0,1264.0,1260.0,14.0,1.0,8.062258,54.0,3.0,54.083269,0.055499,3.179830,0.118569,6.793500,0.365993,20.969862,2916.0,3.988984,True,True,False,False,True,False,False,True,False,False
11056,228,centreBounce,goal,1912.0,4,1519,1560.0,Geelong,Geelong,Jeremy Cameron,Jeremy_Cameron,Kick,41.0,21.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,131.0,45.0,86.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,41.0,49.0,49.0,25.0,21.0,17.0,17.0,33.0,1560.0,1535.0,1532.0,1531.0,41.0,25.0,8.944272,39.0,21.0,44.294469,0.493941,28.300756,0.127885,7.327246,0.400462,22.944787,1521.0,3.663562,True,True,False,False,True,False,False,True,False,False


In [43]:
df_modelling[['ballUp', 'centreBounce', 'kickIn', 'possGain', 'throwIn']] = pd.get_dummies(df_modelling['Initial_State'])

In [44]:
training_data = df_modelling[df_modelling[RESPONSE+"TrainingSet"]]
test_data = df_modelling[df_modelling[RESPONSE+"TestSet"]]
cal_data = df_modelling[df_modelling[RESPONSE+"ValidationSet"]]

In [45]:
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]
X_cal, y_cal = cal_data.drop(columns=[RESPONSE]), cal_data[RESPONSE]

In [46]:
y_train.mean(), y_test.mean(), y_cal.mean()

(0.12507066139061618, 0.1252260397830018, 0.12492933860938384)

In [47]:
X_test.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Behind,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,GoalTrainingSet,GoalTestSet,GoalValidationSet,BehindTrainingSet,BehindTestSet,BehindValidationSet,MissTrainingSet,MissTestSet,MissValidationSet,ballUp,centreBounce,kickIn,possGain,throwIn
2,34,possGain,behind,354.0,1,1015,1080.0,Sydney,Sydney,Josh P. Kennedy,Josh_P._Kennedy,Kick,45.0,16.0,ineffective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,1,1,0,1,18.0,8.0,10.0,Kick,Uncontested Mark,Kick,Handball Received,45.0,48.0,34.0,34.0,16.0,13.0,1.0,-2.0,1080.0,1046.0,1045.0,1045.0,65.0,34.0,4.242641,33.0,16.0,36.674242,0.451453,25.866357,0.158231,9.065951,0.505731,28.976251,1089.0,3.496508,True,False,False,True,False,False,True,False,True,False,0,0,0,1,0
4,59,centreBounce,goal,578.0,2,0,43.0,Sydney,Sydney,Callum Mills,Callum_Mills,Kick,33.0,12.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,0,6,20.0,16.0,4.0,Kick,Uncontested Mark,Kick Inside 50 Result,Kick Into F50,33.0,40.0,40.0,9.0,12.0,11.0,11.0,-8.0,43.0,21.0,20.0,20.0,43.0,22.0,7.071068,45.0,12.0,46.572524,0.260602,14.931417,0.133410,7.643825,0.416017,23.835998,2025.0,3.806662,True,True,False,False,True,False,False,False,True,False,0,1,0,0,0
6,72,possGain,goal,696.0,2,462,506.0,Sydney,Sydney,Isaac Heeney,Isaac_Heeney,Kick,47.0,35.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,0,6,20.0,28.0,-8.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,47.0,53.0,53.0,42.0,35.0,27.0,27.0,41.0,506.0,468.0,466.0,465.0,44.0,38.0,10.000000,31.0,35.0,46.754679,0.845930,48.468229,0.091187,5.224603,0.284262,16.287036,961.0,3.433987,True,True,False,False,False,False,True,False,True,False,0,0,0,1,0
10,108,possGain,goal,982.0,2,1782,1859.0,Sydney,Sydney,Sam Reid,Sam_Reid,Kick,40.0,36.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,0,6,50.0,46.0,4.0,Kick,Uncontested Mark,Kick,Uncontested Mark,40.0,44.0,43.0,50.0,36.0,32.0,48.0,40.0,1859.0,1835.0,1833.0,1814.0,77.0,24.0,5.656854,38.0,36.0,52.345009,0.758378,43.451842,0.089092,5.104600,0.275545,15.787586,1444.0,3.637586,True,True,False,False,False,True,False,False,True,False,0,0,0,1,0
12,132,possGain,goal,1168.0,3,422,469.0,Sydney,Sydney,Logan McDonald,Logan_McDonald,Kick,47.0,-4.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,0,6,57.0,58.0,-1.0,Kick,Mark On Lead,Kick,Gather,47.0,55.0,30.0,22.0,-4.0,-4.0,2.0,6.0,469.0,434.0,432.0,430.0,47.0,35.0,8.000000,31.0,4.0,31.256999,0.128323,7.352379,0.205222,11.758330,0.672664,38.540807,961.0,3.433987,True,True,False,False,True,False,False,False,True,False,0,0,0,1,0


Preprocess Data

In [48]:
# preprocessor = DataPreprocessor()
# preprocessor.fit(X_train)

# X_train_preproc = preprocessor.transform(X_train)
# X_test_preproc = preprocessor.transform(X_test)

In [49]:
X_train_preproc = X_train[FEATURES]
X_test_preproc = X_test[FEATURES]
X_cal_preproc = X_cal[FEATURES]

In [50]:
X_train_preproc.shape, X_test_preproc.shape

((7076, 5), (2212, 5))

In [51]:
X_train_preproc.head()

,x0,y0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle
1,52.0,35.0,43.600459,0.931882,0.088007
3,59.0,27.0,33.015148,0.957589,0.112618
5,52.0,36.0,44.407207,0.945311,0.084822
7,38.0,5.0,40.311289,0.124355,0.158537
8,42.0,-2.0,36.055513,0.055499,0.178638


In [52]:
X_test_preproc.head()

,x0,y0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle
2,45.0,16.0,36.674242,0.451453,0.158231
4,33.0,12.0,46.572524,0.260602,0.133410
6,47.0,35.0,46.754679,0.845930,0.091187
10,40.0,36.0,52.345009,0.758378,0.089092
12,47.0,-4.0,31.256999,0.128323,0.205222


Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [53]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train, monotonicity_constraints=MONOTONE_CONSTRAINTS)

In [54]:
xgb_tuner.training_data.head()

,x0,y0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle
1,52.0,35.0,43.600459,0.931882,0.088007
3,59.0,27.0,33.015148,0.957589,0.112618
5,52.0,36.0,44.407207,0.945311,0.084822
7,38.0,5.0,40.311289,0.124355,0.158537
8,42.0,-2.0,36.055513,0.055499,0.178638


In [55]:
xgb_tuner.tune_hyperparameters()

[I 2023-07-11 15:25:10,756] A new study created in memory with name: no-name-32cec863-0969-4ac0-a70d-099e6bd43d1b
[I 2023-07-11 15:25:10,797] Trial 0 finished with value: 0.6348264420203579 and parameters: {'max_depth': 2, 'min_child_weight': 58, 'eta': 0.011008701412973549, 'gamma': 0.013420685248281482, 'lambda': 0.002935785079707382, 'alpha': 0.06719580458583402, 'subsample': 0.28828516884174893, 'colsample_bytree': 0.5162459994314914}. Best is trial 0 with value: 0.6348264420203579.
[I 2023-07-11 15:25:10,824] Trial 1 finished with value: 0.31316958474559065 and parameters: {'max_depth': 19, 'min_child_weight': 75, 'eta': 0.7797948059982815, 'gamma': 0.1119264940348594, 'lambda': 0.42991994820208806, 'alpha': 0.07443298752126767, 'subsample': 0.49430900551480633, 'colsample_bytree': 0.36834399906142756}. Best is trial 1 with value: 0.31316958474559065.
[I 2023-07-11 15:25:10,848] Trial 2 finished with value: 0.589227064106427 and parameters: {'max_depth': 19, 'min_child_weight': 78

Number of finished trials:  1000
Best trial:
  Value: 0.276462221044005
  Params: 
    max_depth: 5
    min_child_weight: 91
    eta: 0.8054932263328902
    gamma: 0.001692659040608997
    lambda: 0.22642848948045613
    alpha: 0.0013623546862307986
    subsample: 0.7750525506963826
    colsample_bytree: 0.6479794100687422


In [56]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 5,
 'min_child_weight': 91,
 'eta': 0.8054932263328902,
 'gamma': 0.001692659040608997,
 'lambda': 0.22642848948045613,
 'alpha': 0.0013623546862307986,
 'subsample': 0.7750525506963826,
 'colsample_bytree': 0.6479794100687422}

Training Model - SuperXGBClassifier class for training and predictions

In [57]:
params['objective'] = OptunaXGBParamGrid.error
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1
params['monotone_constraints'] = MONOTONE_CONSTRAINTS

In [58]:
super_xgb = SuperXGBClassifier(X_train = X_train_preproc, 
                               y_train = y_train, 
                               X_test = X_test_preproc, 
                               y_test = y_test,
                               X_cal = X_cal_preproc,
                               y_cal = y_cal,
                               params = params)

In [59]:
super_xgb.fit()

[0]	validation_0-logloss:0.38861	validation_1-logloss:0.38365
[1]	validation_0-logloss:0.33444	validation_1-logloss:0.32944
[2]	validation_0-logloss:0.31940	validation_1-logloss:0.31307
[3]	validation_0-logloss:0.31615	validation_1-logloss:0.30808
[4]	validation_0-logloss:0.31477	validation_1-logloss:0.30567
[5]	validation_0-logloss:0.31410	validation_1-logloss:0.30638
[6]	validation_0-logloss:0.31341	validation_1-logloss:0.30580
[7]	validation_0-logloss:0.31286	validation_1-logloss:0.30498
[8]	validation_0-logloss:0.31196	validation_1-logloss:0.30544
[9]	validation_0-logloss:0.31171	validation_1-logloss:0.30664
[10]	validation_0-logloss:0.31158	validation_1-logloss:0.30547
[11]	validation_0-logloss:0.31162	validation_1-logloss:0.30432
[12]	validation_0-logloss:0.31165	validation_1-logloss:0.30434
[13]	validation_0-logloss:0.31168	validation_1-logloss:0.30396
[14]	validation_0-logloss:0.31110	validation_1-logloss:0.30318
[15]	validation_0-logloss:0.31083	validation_1-logloss:0.30279
[1

In [60]:
super_xgb.xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6479794100687422, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.001692659040608997, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.8054932263328902, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=91, missing=nan,
              monotone_constraints={'Angle_to_Middle_Goal': 1,
                                    'Distance_to_Middle_Goal': 1,
                                    'Visible_Goal_Angle': -1, 'x0': 1},
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [61]:
super_xgb.xgb_model.get_booster().feature_names

['x0',
 'y0',
 'Distance_to_Middle_Goal',
 'Angle_to_Middle_Goal',
 'Visible_Goal_Angle']

In [62]:
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

In [63]:
train_probas = super_xgb.predict_proba(X_train_preproc)[:, 1]
test_probas = super_xgb.predict_proba(X_test_preproc)[:, 1]
cal_probas = super_xgb.predict_proba(X_cal_preproc)[:, 1]

In [64]:
super_xgb.calibrate()

In [65]:
train_cal_probas = super_xgb.predict_proba(X_train_preproc, calibrate=True)
test_cal_probas = super_xgb.predict_proba(X_test_preproc, calibrate=True)
cal_cal_probas = super_xgb.predict_proba(X_cal_preproc, calibrate=True)

Check Average Predictions

In [66]:
train_probas.mean(), training_data[RESPONSE].mean(), train_cal_probas.mean()

(0.12599026, 0.12507066139061618, 0.12424147140654518)

In [67]:
test_probas.mean(), test_data[RESPONSE].mean(), test_cal_probas.mean()

(0.12564427, 0.1252260397830018, 0.1238175550054479)

In [68]:
cal_probas.mean(), cal_data[RESPONSE].mean(), cal_cal_probas.mean()

(0.12665357, 0.12492933860938384, 0.12492924397296047)

Export model

In [69]:
super_xgb.export_model(model_output_path + "/" + model_file_name + ".joblib")

Export data and predictions

In [70]:
train_info = training_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
train_export = pd.concat([train_info, X_train_preproc.reset_index(drop=True)], axis=1)
train_export['xgb_preds'] = train_preds
train_export['xgb_probas'] = train_probas
train_export['xgb_probas_cal'] = train_cal_probas
train_export.to_csv(prediction_output_path + 'train_predictions_' + model_file_name + '.csv', index = False)
train_export.head()

,Match_ID,Chain_Number,Order,Miss,x0,y0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,28,295.0,0,52.0,35.0,43.600459,0.931882,0.088007,0,0.185089,0.189333
1,202101_BrisbaneLions_Sydney,53,537.0,0,59.0,27.0,33.015148,0.957589,0.112618,0,0.150782,0.152445
2,202101_BrisbaneLions_Sydney,62,621.0,0,52.0,36.0,44.407207,0.945311,0.084822,0,0.235237,0.241626
3,202101_BrisbaneLions_Sydney,76,723.0,0,38.0,5.0,40.311289,0.124355,0.158537,0,0.020630,0.015300
4,202101_BrisbaneLions_Sydney,84,799.0,0,42.0,-2.0,36.055513,0.055499,0.178638,0,0.011555,0.007635


In [71]:
test_info = test_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
test_export = pd.concat([test_info, X_test_preproc.reset_index(drop=True)], axis=1)
test_export['xgb_preds'] = test_preds
test_export['xgb_probas'] = test_probas
test_export['xgb_probas_cal'] = test_cal_probas
test_export.to_csv(prediction_output_path + 'test_predictions_' + model_file_name + '.csv', index = False)
test_export.head()

,Match_ID,Chain_Number,Order,Miss,x0,y0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,34,354.0,0,45.0,16.0,36.674242,0.451453,0.158231,0,0.019428,0.014240
1,202101_BrisbaneLions_Sydney,59,578.0,0,33.0,12.0,46.572524,0.260602,0.133410,0,0.080402,0.075776
2,202101_BrisbaneLions_Sydney,72,696.0,0,47.0,35.0,46.754679,0.845930,0.091187,0,0.227630,0.233838
3,202101_BrisbaneLions_Sydney,108,982.0,0,40.0,36.0,52.345009,0.758378,0.089092,0,0.385912,0.383841
4,202101_BrisbaneLions_Sydney,132,1168.0,0,47.0,-4.0,31.256999,0.128323,0.205222,0,0.010350,0.006688


Save preprocessor

In [ ]:
# joblib.dump(preprocessor, preprocessor_output_path + preprocessor_file_name + ".joblib")